# Data loader example

This is example code that demonstrates use of our custom data loader for the POCOVID-Net training data. Much of this code is based on snippets from the the official PyTorch tutorial on [custom datasets, dataloaders, and transforms](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html). 

In [52]:
import os
import torch
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

## Subclass `torch.utils.data.Dataset`

In [53]:
class PocovidDataset(Dataset):
  """Subclass of Dataset for POCOVID-Net data"""
  
  def __init__(self, root_dir, transform=None):
      """
      Args:
          root_dir (string): Directory with all the images.
          transform (callable, optional): Optional transform to be applied
              on a sample.
      """
      self.root_dir = root_dir
      self.transform = transform
      self.covid_dir = root_dir + '/' + 'covid'
      self.pneu_dir = root_dir + '/' + 'pneumonia'
      self.regular_dir = root_dir + '/' + 'regular'
      
      self.covid_class = 0
      self.pneu_class = 1
      self.regular_class = 2

      # Modified code snippet from Daniel Stutzbach: https://stackoverflow.com/a/2632251
      dir_items = lambda d: [d + "/" + name for name in os.listdir(d) if os.path.isfile(d + "/" + name)]

      covid_items = dir_items(self.covid_dir)
      pneu_items = dir_items(self.pneu_dir)
      regular_items = dir_items(self.regular_dir)
      
      num_covid = len(covid_items)
      num_pneu = len(pneu_items)
      num_regular = len(regular_items)
      
      self.img_info = []
      for covid_filename in covid_items:
        self.img_info.append((covid_filename,self.covid_class))
      for pneu_filename in pneu_items:
        self.img_info.append((pneu_filename,self.pneu_class))
      for regular_filename in regular_items:
        self.img_info.append((regular_filename,self.regular_class)) 

      self.transform = transform
      self.num_images = num_covid + num_pneu + num_regular
  
  def __len__(self):
    return self.num_images
  
  def __getitem__(self,idx):
    img_name, img_class = self.img_info[idx]
    image = io.imread(img_name)
    sample = {'image': image, 'class': img_class} 
    
    if self.transform:
      sample = self.transform(sample)

    return sample

## Define transformations
These are the same or very similar to transformations defined in the official PyTorch tutorial on [custom datasets, dataloaders, and transforms](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html)

In [69]:
# TODO: Copy Rescale from the tutorial, and create a "rotate" tranform and a "translate" transform
# because the paper augments data in place by rotating up to 10 degrees, and translating up to 10 
# degrees

# Slightly modified from official PyTorch tutorial: 
# https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#transforms
class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, img_cls = sample['image'], sample['class']

        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = transform.resize(image, (new_h, new_w))

        return {'image': img, 'class': img_cls}

# From official PyTorch tutorial: 
# https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#transforms
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, img_cls = sample['image'], sample['class']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image),
                'class': img_cls}

## Create an instance of `PocovidDataset`, applying transformations "in place"

For the purposes of this example, I am assuming this notebook is being run from the `code/app` directory in this repository. If you use a different working directory, you will need to adjust the `root_dir` argument to the `PocovidDataset` constructor accordingly. 

In [70]:
os.getcwd() # get current working directory

'/home/nathan/Code/covid19_pocus_ultrasound_pytorch/code/app'

In [71]:
transformed_dataset = PocovidDataset(root_dir='../data/pocus_images',
                                     transform=transforms.Compose([
                                       Rescale(224),
                                       ToTensor()
                                       ]))

## Iterating through the dataset

### Option A: Using a `for` loop
First, let's iterate using a simple `for` loop. This works, but may not be as powerful / as efficient as using a `DataLoader` (see Option B).

In [72]:
for i in range(len(transformed_dataset)):
    sample = transformed_dataset[i]

    print(i, sample['image'].size(), sample['class'])

    if i == 3:
        break

0 torch.Size([3, 224, 224]) 0
1 torch.Size([3, 224, 224]) 0
2 torch.Size([3, 224, 224]) 0
3 torch.Size([3, 224, 224]) 0


### Option B: Using `torch.utils.data.DataLoader`
**TODO:** get the commented code below to actually work.

In [42]:
# dataloader = DataLoader(covid_data, batch_size=4, shuffle=True, num_workers=0)

In [41]:
# FIXME: 0, 1, and 2 are hardcoded both here and where we set self.covid_class, self.pneu_class, and 
# self.regular_class in the __init__ function for PocovidDataset
# lbl_map = {0:"COVID",1:"Pneumonia",2:"Regular"} 

# # based on show_landmarks_batch in the official PyTorch dataloader tutorial
# def show_labeled_images_batch(sample_batched):
#   """Show image with class labels for a batch of samples."""
#   images_batch, classes_batch = sample_batched['image'], sample_batched['class']
#   print(images_batch)
#   print(classes_batch)

In [44]:
# for i_batch, sample_batched in enumerate(dataloader):
#     print(i_batch, sample_batched['image'].size(),
#           sample_batched['class'].size())

#     # observe 4th batch and stop.
#     if i_batch == 3:
#         plt.figure()
#         show_landmarks_batch(sample_batched)
# #         plt.axis('off')
# #         plt.ioff()
# #         plt.show()
#         break

RuntimeError: stack expects each tensor to be equal size, but got [1080, 1080, 3] at entry 0 and [980, 1350, 3] at entry 1